In [ ]:
import numpy as np
from scipy.optimize import least_squares as lsq
import robustsp as rsp

def arma_est_bip_m(x, p, q, beta_hat_s, a_sc_final):
    N = len(x);
    x = np.array(x)
    F_mm = lambda beta: sp.sqrt(1/(N-p)*sp.sum(rsp.muler_rho2(rsp.arma_resid(x/a_sc_final, beta, p, q))))
    F_bip_mm = lambda beta: sp.sqrt(1/(N-p)*sp.sum(rsp.muler_rho2(rsp.bip_resid(x/a_sc_final, beta, p, q))))
    
    beta_arma_mm = lsq(F_mm, beta_hat_s,xtol=5*1e-5,ftol=5*1e-5,method='lm')[0]
    beta_bip_mm = lsq(F_bip_mm, beta_hat_s,xtol=5*1e-5,ftol=5*1e-5,method='lm')[0]
    
    a_rho2_mm = 1/(N-p)*np.sum(muler_rho2(rsp.arma_resid(x/a_sc_final,beta_arma_mm,p,q)))
    a_bip_rho2_mm = 1/(N-p)*np.sum(muler_rho2(rsp.bip_resid(x/a_sc_final,beta_bip_mm,p,q)))
    
    beta_hat = beta_arma_mm if a_rho2_mm<a_bip_rho2 else beta_bip_mm
    
    # Output the results
    phi_bip_mm = -beta_hat[:p] if p>0 else []
    theta_bip_mm = -beta_hat[p:] if q>0 else []
    
    return phi_bip_mm, theta_bip_mm

In [58]:
import numpy as np
import scipy as sp

def arma_resid(xx, beta_hatx, p, q):
    x = np.array(xx)
    beta_hat = np.array(beta_hatx)
    if p>0:
        phi_hat = beta_hat[:p]
    else:
        phi_hat = []
        
    if q>0:
        theta_hat = beta_hat[p:]
    else:
        theta_hat = []
        
    N = len(x)
    r = max(p,q)
    a = np.zeros(N)
    
    if r==0:
        a = np.array(x)
    elif p>=1 and q >=1:
        for ii in range(r,N):
            # ARMA residuals
            xArr = x[ii-1::-1] if ii-p-1 < 0 else x[ii-1:ii-p-1:-1]
            aArr = a[ii-1::-1] if ii-q-1 < 0 else a[ii-1:ii-q-1:-1]
            a[ii] = x[ii]-phi_hat@xArr+theta_hat@aArr
    elif p==0 and q>=1:
        # MA model
        for ii in range(r,N):
            aArr = a[ii-1::-1] if ii-q-1 < 0 else a[ii-1:ii-q-1:-1]
            a[ii] = x[ii] + theta_hat@aArr
    elif p>=1 and q == 0:
        # AR models
        for ii in range(r,N):
            # AR residuals
            xArr = x[ii-1::-1] if ii-p-1 < 0 else x[ii-1:ii-p-1:-1]
            a[ii] = x[ii]-phi_hat@xArr
    return a
'''
>> arma_resid([1 2 3 4]', [1,2,3],3,0)

ans =

     0
     0
     0
    -6
'''
pr(arma_resid([1, 2, 3 ,4], [1,2,3],3,0))

'''
>> arma_resid([1 2 3 4]', [1,2,3],0,3)

ans =

     0
     0
     0
     4
'''
pr(arma_resid([1, 2, 3 ,4], [1,2,3],0,3))

arma_resid([1, 2, 3 ,4], [1,2,1,2,3],2,3)

[ 0.  0.  0. -6.]
[0. 0. 0. 4.]


array([ 0.,  0.,  0., -3.])

In [84]:
import numpy as np
import scipy as sp
import robustsp as rsp

def bip_resid(xx, beta_hatx, p, q):
    x = np.array(xx)
    beta_hat = np.array(beta_hatx)
    
    phi_hat = beta_hat[:p] if p>0 else []
    theta_hat = beta_hat[p:] if q>0 else []
    
    N = len(x)
    r = max(p,q)
    a_bip = np.zeros(N)
    x_sc = rsp.m_scale(x)
    kap2 = 0.8724286
    
    if np.sum(np.abs(np.roots(np.array([1, *phi_hat*-1])))>1)\
    or np.sum(np.abs(np.roots(np.array([1, *theta_hat])))>1):
        sigma_hat = x_sc
        a_bip = np.array(x)
    else:
        lamb = rsp.ma_infinity(phi_hat, -theta_hat, 100)
        sigma_hat = np.sqrt(x_sc**2 / (1+kap2*np.sum(lamb**2)))
        
        if r == 0:
            a_bip = np.array(x)
        else:
            if p>=1 and q>=1:
                # ARMA Models
                #for ii in range(r,N):
                while r+1 != N:
                    ii = r
                    # BIP-ARMA residuals
                    xArr = x[ii-1::-1] if ii-p-1 < 0 else x[ii-1:ii-p-1:-1]
                    abArr = a_bip[ii-1::-1] if ii-p-1 < 0 else a_bip[ii-1:ii-p-1:-1]
                    aqArr = a_bip[ii-1::-1] if ii-q-1 < 0 else a_bip[ii-1:ii-q-1:-1]
                    a_bip[ii] = x[ii]-phi_hat@(xArr-abArr+sigma_hat*rsp.eta(abArr/sigma_hat))+sigma_hat*theta_hat@rsp.eta(aqArr/sigma_hat)
                    r +=1
            elif p==0 and q>=1:
                # MA models
                for ii in range(r,N):
                    # BIP-MA residuals
                    aArr = a_bip[ii-1::-1] if ii-q-1 < 0 else a_bip[ii-1:ii-q-1:-1]
                    a_bip[ii]=x[ii]+theta_init*sigma_hat*rsp.eta(aArr/sigma_hat)
            elif p>=1 and q==0:
                # AR models
                for ii in range(r,N):
                    # BIP-AR residuals
                    xArr = x[ii-1::-1] if ii-p-1 < 0 else x[ii-1:ii-p-1:-1]
                    aArr = a_bip[ii-1::-1] if ii-p-1 < 0 else a_bip[ii-1:ii-p-1:-1]
                    a_bip[ii] = x[ii]+phi_hat@(xArr-aArr)+sigma_hat*rsp.eta(aArr/sigma_hat)

    return a_bip[p:]
  
# stimmt bip_resid([1,2,3,4], [1,2,1,2,3],2,3) # 3,4
# stimmt bip_resid([1,2,3,4,5], [1,2,1,2,3],2,3) # 3,4,5
# stimmt bip_resid([1,2,3,4,5], [2,3],2,0)
# stimmt bip_resid([1,2,3,4,5], [2,3],0,2)

array([1, 2, 3, 4, 5])

In [1]:
a = [1,2,3,4,5]
a[:1]

[1]